# Project

## Goals
* 

## Topics to explore: 
* How likely are people to stick with their parents' political alignment? 
* If one of your parents is a democrat, and the other is a republican, does that make you more likely to be apolitical, a centrist, or pick one or the other? If you are going to pick one or the other, which one is more likely to be picked? What trends are there if any? 
* Does more politically active parents lead to more politically active children?
* How likely are you to have political views that align with your parents political party?
* Is there a correlation between socio-economic class and political involvement?

## Variables:
* “Feeling thermometer” variables (e.g. policemen, political parties, people on welfare, feminists, people seeking to protect the environment”
* “Women’s equal role” - Should women have equal role in the household as the man (7-pt scale)?
* “During the past 12 months, have you worked with other people to deal with some issues facing your community?”
* “Did you give any money to a political party this year?”
* “Some people don’t pay much attention to political campaigns…”
* “Do you remember when you were growing up whether your  father was very much interested in politics, somewhat interested , or he didn’t pay much attention?”
* Same as above but for mothers
* “Did your father participate in elections?”
* Same as above but for mothers
* Did you give any money to a political party this year?

## To Do 
* Download data
* [Load data](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_stata.html?highlight=dta)
* Learn about weighting
* Load variables

In [1]:
import pandas as pd
import os

Unpack the tar file if the data file is not already uncompressed on this machine:

In [5]:
if not os.path.exists('anes_timeseries_cdf_rawdata.txt'):
    !tar -xvf data.tar.bz2

Import the data and save as dataframe:

In [57]:
df = pd.read_csv('anes_timeseries_cdf_rawdata.txt', delimiter=',', low_memory=False, na_values=' ')
df

,Version,VCF0004,VCF0006,VCF0006a,VCF0009x,VCF0010x,VCF0011x,VCF0009y,VCF0010y,VCF0011y,...,VCF9272,VCF9273,VCF9274,VCF9275,VCF9277,VCF9278,VCF9279,VCF9280,VCF9281,VCF9282
0,ANES_CDF_VERSION:2019-Sep-10,1948,1001,19481001,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
1,ANES_CDF_VERSION:2019-Sep-10,1948,1002,19481002,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
2,ANES_CDF_VERSION:2019-Sep-10,1948,1003,19481003,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
3,ANES_CDF_VERSION:2019-Sep-10,1948,1004,19481004,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
4,ANES_CDF_VERSION:2019-Sep-10,1948,1005,19481005,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59939,ANES_CDF_VERSION:2019-Sep-10,2016,5086,20165086,0.0,0.0,0.0,0.818,0.818,0.818,...,2,1,,3,1,2,1,1,1,1
59940,ANES_CDF_VERSION:2019-Sep-10,2016,5087,20165087,0.0,0.0,0.0,1.280,1.280,1.280,...,3,6,,2,-9,-9,1,2,1,1
59941,ANES_CDF_VERSION:2019-Sep-10,2016,5088,20165088,0.0,0.0,0.0,3.788,3.788,3.788,...,1,5,,2,-9,-9,1,2,1,1
59942,ANES_CDF_VERSION:2019-Sep-10,2016,5089,20165089,0.0,0.0,0.0,1.162,1.162,1.162,...,2,2,,3,2,2,1,2,1,1


Replace na_values string NaN

In [36]:
df.replace(type(str), 0)

,Version,VCF0004,VCF0006,VCF0006a,VCF0009x,VCF0010x,VCF0011x,VCF0009y,VCF0010y,VCF0011y,...,VCF9272,VCF9273,VCF9274,VCF9275,VCF9277,VCF9278,VCF9279,VCF9280,VCF9281,VCF9282
0,ANES_CDF_VERSION:2019-Sep-10,1948,1001,19481001,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
1,ANES_CDF_VERSION:2019-Sep-10,1948,1002,19481002,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
2,ANES_CDF_VERSION:2019-Sep-10,1948,1003,19481003,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
3,ANES_CDF_VERSION:2019-Sep-10,1948,1004,19481004,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
4,ANES_CDF_VERSION:2019-Sep-10,1948,1005,19481005,1.0,1.0,1.0,1.000,1.000,1.000,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59939,ANES_CDF_VERSION:2019-Sep-10,2016,5086,20165086,0.0,0.0,0.0,0.818,0.818,0.818,...,2,1,,3,1,2,1,1,1,1
59940,ANES_CDF_VERSION:2019-Sep-10,2016,5087,20165087,0.0,0.0,0.0,1.280,1.280,1.280,...,3,6,,2,-9,-9,1,2,1,1
59941,ANES_CDF_VERSION:2019-Sep-10,2016,5088,20165088,0.0,0.0,0.0,3.788,3.788,3.788,...,1,5,,2,-9,-9,1,2,1,1
59942,ANES_CDF_VERSION:2019-Sep-10,2016,5089,20165089,0.0,0.0,0.0,1.162,1.162,1.162,...,2,2,,3,2,2,1,2,1,1


Weight

First focus on finding the correlation between parent political party and political party of the child.
Need variables for:
1. Father's political party
- When you were growing up did your father think of himself mostly as a Democrat, as a Republican, or what? - VCF0306 (1. Democrat, 2. Indepedent, Republican, 4. Other, never voted, din't get into politics, DK (1988), 9. DK)
2. Mother's political party
- When you were growing up did your mother think of himself mostly as a Democrat, as a Republican, or what? - VCF0306 (1. Democrat, 2. Indepedent, Republican, 4. Other, never voted, din't get into politics, DK, 9. DK)
VCF0307
3. Political affiliation
- Major party thermometer: VCF0291 (2. Most Republican, 50. Neutral, 99. Most Democratic)
- Genereally speaking, do you usually think of yourself as Republican, a Democrat, an Independent, or what? Do you think of yourself as closer to the Republican or Demucratic party? VCF0301 (1. Strong Democrat, 2. Weak Democrat, 3. Independent - Democrat, 4. Independent -  Independent, 5. Independent - Republican, 6. Weak Republican, 7. Strong Republican, 0. NA)
4. Respondent gender - VCF0104 (1 - male, 2 -female, 3 - other, 0 - NA, INAP - question not used), Weight: VCF0009x/VCF0009y/VCF0009z

In [59]:
ts = df['VCF0306']
ts.sum()

53824.0

In [48]:
df['VCF0307']

0         
1         
2         
3         
4         
        ..
59939     
59940     
59941     
59942     
59943     
Name: VCF0307, Length: 59944, dtype: object

In [49]:
df['VCF0301']

0         
1         
2         
3         
4         
        ..
59939    1
59940    1
59941    2
59942    1
59943    6
Name: VCF0301, Length: 59944, dtype: object

In [53]:
df['VCF0104']

0        1
1        2
2        2
3        2
4        1
        ..
59939    2
59940    1
59941    1
59942    2
59943    1
Name: VCF0104, Length: 59944, dtype: int64

Questions:
* Should I be doing any sort of weighting?

1. Father's Political Interest
- "Do you remember when you were growing up whether your father was very much interested in politics, somewhat interested, or didn't he pay much attention to it?" (1. Didn't pay much attention, 2. Somewhat interested, 3. Very much interested / 9. DK, 0. NA, INAP no question not worn)
- VCF0308
2. Mother's Political Interest
- "Now how about your mother? When you were growing up was she very much interested in politics, somewhat interested, or didn't he pay much attention to it?" (1. Didn't pay much attention, 2. Somewhat interested, 3. Very much interested / 9. DK, 0. NA, INAP no question not worn)
- VCF0309
3. Your Political Interest
- "Some people don't pay much attention to political campaigns. How about you, would you say that you have been/were very much interested, somewhat interested, or not much interested in political campaigns this year?" (1. Didn't pay much attention, 2. Somewhat interested, 3. Very much interested / 9. DK, 0. NA, INAP no question not worn)
- VCF0310 


Feeling Thermeter Political Parties:

Democrats:
* 00-96 Degrees as coded
* 97-1000 Degrees
* 98 - DK (exc. 1964-1968: see VCF0201 note)
* 99 - NA (1972)
* INAP - question not used

Weight: VCF0009x/VCF0009y/VCF0009z

pg. 90

Republicans: